In [ ]:
from fastai.vision.all import *

labels = pd.read_csv("../input/dog-breed-identification/labels.csv")
labels

# Get the data

split the data into training and validation sets

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
train_ids, valid_ids = next(split.split(labels, labels["breed"]))
labels["is_valid"] = [i in valid_ids for i in range(len(labels))]

labels["id"] = labels["id"].apply(lambda x: x + ".jpg")

get images, resize each image, add data augmentation and normalize image tensors

In [ ]:
path = "../input/dog-breed-identification/train"

dls = ImageDataLoaders.from_df(labels, path,
                               item_tfms=Resize(460, method="squeeze"),
                               batch_tfms=[*aug_transforms(size=300),
                                           Normalize.from_stats(*imagenet_stats)],
                               bs=32, valid_col="is_valid")
dls.show_batch()

# Extracting feature using various pretrained CNN

downloading trained CNNs as feature extractor: inception, resnet50

In [ ]:
from torchvision.models import inception_v3, mobilenet_v2

inception = inception_v3(pretrained=True, aux_logits=False)
inception = nn.Sequential(*list(inception.children())[:-2],
                          nn.Flatten()).eval()

In [ ]:
resnet = nn.Sequential(*list(resnet50(pretrained=True).children())[:-1], 
                          nn.Flatten()).eval()

In [ ]:
mobile = nn.Sequential(*list(mobilenet_v2(pretrained=True).children())[:-1],
                       nn.AdaptiveAvgPool2d((1,1)),
                       nn.Flatten()).eval()

combine these CNNs and add a classifier

In [ ]:
class NeuralNet(Module):
    def __init__(self, extractors, hidden_size, vocab_size, device):
        
        self.extractors = extractors
        for conv in self.extractors:
            conv.to(device)
                  
        self.classifier = nn.Sequential(
            nn.BatchNorm1d(hidden_size),
            nn.Dropout(0.25),
            nn.Linear(hidden_size, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Dropout(0.5),
            nn.Linear(1024, vocab_size)
        )
        
    def forward(self, x):
        
        features = torch.cat([conv(x) for conv in self.extractors], dim=1)
        
        return self.classifier(features)

# Train our model

In [ ]:
extractors = [inception, resnet, mobile]
hidden_size = 2048 + 2048 + 1280
device = "cuda" if torch.cuda.is_available() else "cpu"
model = NeuralNet(extractors, hidden_size, len(dls.vocab), device)

In [ ]:
weights = [labels.shape[0] / (120 * labels["breed"].value_counts()[breed]) for breed in dls.vocab]
weights = tensor(weights, device=device)

In [ ]:
learn = Learner(dls, model, metrics=accuracy, path=".").to_fp16()
learn.lr_find()

In [ ]:
learn.fit_one_cycle(3, 1e-3)

# Get predictions

In [ ]:
torch.cuda.empty_cache()

In [ ]:
test_files = get_image_files("../input/dog-breed-identification/test")
test_dl = dls.test_dl(test_files, bs=32)

In [ ]:
preds, targs = learn.get_preds(dl=test_dl)

In [ ]:
sub = pd.DataFrame({"id":test_files.map(lambda x:x.stem)})
sub[list(dls.vocab)] = preds
sub.to_csv("submission.csv", index=False)